<a href="https://colab.research.google.com/github/Gsus329/Machine-Learning/blob/main/Proyecto__1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Obtener datos**

¡Bienvenido a Machine Learning Housing Corp.! Su tarea es predecir el valor medio de las viviendas en los distritos de California, teniendo en cuenta una serie de características de estos distritos.

# Descargar Datos

Código para descargar los datos desde necesarios para nuestro sistema Machine Learning

In [ ]:
from pathlib import Path
import pandas as pd
import tarfile
import urllib.request

def load_housing_data():
    tarball_path = Path("datasets/housing.tgz")
    if not tarball_path.is_file():
        Path("datasets").mkdir(parents=True, exist_ok=True)
        url = "https://github.com/Gsus329/Machine-Learning/raw/main/Datos/housing.tgz"
        urllib.request.urlretrieve(url, tarball_path)
        with tarfile.open(tarball_path) as housing_tarball:
            housing_tarball.extractall(path="datasets")
    return pd.read_csv(Path("datasets/housing/housing.csv"))

housing = load_housing_data()